# Load Packages

In [1]:
import os
import pandas as pd
import seaborn as sns
from copulas.visualization import compare_2dv
from copulas.multivariate import GaussianMultivariate

# Load Data

In [2]:
data_nm="weight-height.csv"
base_data_folder="data"

In [3]:
# Download data
data_path=os.path.join("..",base_data_folder,data_nm)
if not os.path.isfile(data_path):
    print("Downloading data ...")
    !kaggle datasets download -d krishnaraj30/weight-and-height-data-outlier-detection -p ../data --unzip
    print("Download done!")
else:
    print("Data already exist")


Data already exist


In [4]:
df=pd.read_csv(data_path)

In [5]:
df

,Gender,Height,Weight
0,Male,73.847017,241.893563
1,Male,68.781904,162.310473
2,Male,74.110105,212.740856
3,Male,71.730978,220.042470
4,Male,69.881796,206.349801
...,...,...,...
9995,Female,66.172652,136.777454
9996,Female,67.067155,170.867906
9997,Female,63.867992,128.475319
9998,Female,69.034243,163.852461


# Fit Copula

In [6]:
copula = GaussianMultivariate()
copula.fit(df[["Height","Weight"]])

/Users/ezratampubolon/PythonProjects/copula/.venv/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:5999: RuntimeWarning: divide by zero encountered in divide
  return c**2 / (c**2 - n**2)


# Outlier Detection

In [9]:
df["prob"]=copula.probability_density(df)
df["outlier"]=df["prob"]<0.001

In [19]:
df.query("outlier==True")

,Gender,Height,Weight,prob,outlier
86,Male,66.596197,208.345694,0.000590,True
1243,Male,62.063836,175.071171,0.000766,True
1317,Male,78.462053,227.342565,0.000075,True
1671,Male,70.059331,225.014368,0.000655,True
1750,Male,74.604668,249.946283,0.000793,True
2014,Male,78.998742,269.989699,0.000212,True
2623,Male,76.842679,211.724166,0.000132,True
2843,Male,70.440713,225.797525,0.000978,True
3585,Male,76.453720,213.732348,0.000726,True
3757,Male,78.621374,245.733783,0.000740,True


In [21]:
sns.catplot(data=df,x="Height",y="Weight", hue="outlier",height=500)